**References**

[BM25 Retriever - LlamaIndex](https://docs.llamaindex.ai/en/stable/examples/retrievers/bm25_retriever/)

[Fusion Retrieval with LlamaIndex - GitHub](https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/fusion_retrieval_with_llamaindex.ipynb)

# Setup

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from dotenv import load_dotenv
import os
from llama_index.core.settings import Settings
from llama_index.core.constants import DEFAULT_TEMPERATURE

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Settings.llm = OpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o-mini", temperature=DEFAULT_TEMPERATURE)
Settings.embed_model = OpenAIEmbedding(model_name="text-embedding-ada-002", api_key=OPENAI_API_KEY)

# Vectorstore

In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

def get_vectorstore() -> ChromaVectorStore:
    """
    Initializes and returns a ChromaVectorStore instance.
    This function creates a persistent client for the Chroma database using the
    directory specified by Directory.VECTORSTORE_DIRECTORY. It then retrieves or
    creates a collection named "transcription_project" within the database and
    returns a ChromaVectorStore instance associated with this collection.

    Returns
    -------
        ChromaVectorStore: An instance of ChromaVectorStore associated with the
        "transcription_project" collection.

    """
    db = chromadb.PersistentClient(path="./vectorstore/")
    chroma_collection = db.get_or_create_collection("transcription_project")
    return ChromaVectorStore(chroma_collection=chroma_collection)

# Index

In [4]:
from llama_index.core import VectorStoreIndex

def get_index() -> VectorStoreIndex:
    """
    Retrieves a VectorStoreIndex instance.
    This function initializes a vector store using the `get_vectorstore` function
    and then creates a `VectorStoreIndex` from the vector store using the specified
    embedding model from the settings.

    Returns
    -------
        VectorStoreIndex: An instance of VectorStoreIndex created from the vector store.

    """
    vectorstore = get_vectorstore()
    return VectorStoreIndex.from_vector_store(
        vectorstore,
        embed_model=Settings.embed_model,
    )

In [5]:
from llama_index.core.response.notebook_utils import display_source_node

def display_retrieved_nodes(retrieved_nodes, source_length=5000):
    """
    Displays the retrieved nodes using the display_source_node function.

    Parameters
    ----------
    retrieved_nodes : list
        List of nodes retrieved by the retriever.
    source_length : int, optional
        The length of the source content to display, by default 5000.
    """
    for node in retrieved_nodes:
        display_source_node(node, source_length=source_length)

In [6]:
question = "Who was Salman Al Farsi a son of?"

In [7]:
top_n = 50

# BM25 Retriever

To use a BM25 retriever with ChromaDB setup, we'll need to ensure that we have access to the text data stored in ChromaDB, as BM25 is a text-based retrieval method. Here's how we can set up and use a BM25 retriever with ChromaDB:

**Access Text Data**: Ensure we can extract the text content from our ChromaDB. This might involve retrieving documents and their associated text fields.

**Initialize BM25 Retriever**: Use the BM25Retriever class to create a retriever that operates on the text data.

**Perform Retrieval**: Execute queries using the BM25 retriever to get relevant documents based on text similarity.

## Accessing All Nodes

In [8]:
vectorstore = get_vectorstore()

In [9]:
# Define batch size
batch_size = 5000
offset = 0
all_docs = []

while True:
    # Retrieve a batch of documents
    # batch = chroma_collection.get(limit=batch_size, offset=offset)
    # documents = batch['documents']
    batch = vectorstore._get(limit=batch_size, offset=offset, where={}) # Get all documents
    documents = batch.nodes
    if not documents:
        break

    all_docs.extend(documents)

    offset += batch_size
    break

In [ ]:
import concurrent.futures

vectorstore = get_vectorstore()

# Define batch size
batch_size = 5000

# Get total document count
total_docs = vectorstore._client._collection.count()

# Generate offset values
offsets = range(0, total_docs, batch_size)

all_docs = []

def fetch_batch(offset):
    """Fetch a batch of documents from ChromaDB."""
    batch = vectorstore._get(limit=batch_size, offset=offset, where={})
    return batch.nodes

# Use ThreadPoolExecutor to fetch batches in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    results = executor.map(fetch_batch, offsets)

# Flatten results
for batch in results:
    all_docs.extend(batch)


In [10]:
print(len(all_docs))

5000


In [12]:
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(all_docs)

In [16]:
docstore.persist("./docstore.json")

In [29]:
docstore_loaded = SimpleDocumentStore.from_persist_path("./docstore.json")

In [30]:
len(docstore_loaded.docs)

166866

In [9]:
vectorstore = get_vectorstore()

In [21]:
index = get_index()

In [10]:
documents = vectorstore._get(limit=None, where={}) # Get all documents

In [11]:
nodes = documents.nodes # Get all nodes

In [12]:
len(nodes), type(nodes[0])

(166862, llama_index.core.schema.TextNode)

## Initializing BM25 Retriever

In [24]:
source_nodes = index.as_retriever(similarity_top_k=10000).retrieve(question)
bm25_source_nodes = [x.node for x in source_nodes]

In [11]:
from llama_index.retrievers.bm25 import BM25Retriever

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=all_docs,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
)

resource module not available on Windows


e:\Capelin\llama-index-test\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
bm25_retriever.persist("./bm25_retriever")

UnicodeEncodeError: 'charmap' codec can't encode character '\u0101' in position 236: character maps to <undefined>

In [25]:
# This is how we can set similarity_top_k

bm25_retriever.similarity_top_k = 50

In [26]:
retrieved_nodes = bm25_retriever.retrieve(question)

In [27]:
len(retrieved_nodes)

50

In [28]:
display_retrieved_nodes(retrieved_nodes)

**Node ID:** b4220de6-90dd-48d8-b8c8-e51d95f8c75e<br>**Similarity:** 7.057149887084961<br>**Text:** ![Cover](images/cover.png)


![IslamicMobility](images/logo-feedbooks-tiny.png)

# Salman Al Muhammadi

## Kamal al-Syyed - XKP

  
**Published:** 2012  
**Tag(s):** companion prophet ali imam shia shiite shiaism husain salman farsi
persian iran syria holy xkp madina makka pagan madain kufa


## Chapter 1 The Son of Islam

It was before noon. Some Muslims sat in Prophet's Mosque waiting for Azan
(call to prayer) to say noon prayer.

Salman entered mosque and greeted his believing brothers.

The Muslims wanted to know the Persian man's tribe. They mentioned their
tribes loudly to let Salman hear them.

One of them said:

_I belong to Tamim's tribe._

Another said:

_I belong to the Quraish._

A third said:

_I belong to al-Aus tribe._

And so on.

But Salman was silent. They wanted to know his tribe. So they asked him:

_Salman, where are you from?_

To teach them the meaning of Islam, Salman answered:

_I'm the son on Islam! I was lost! So, Allah's guided me with Muhammad. I was
poor! So, Allah's made me rich with Muhammad. I was a slave! So, Allah's
released me with Muhammad. This is my tribe!_

The Muslims in mosque kept silent because Salman taught them a lesson of
Islam.<br>

**Node ID:** 242cd2bd-ec3c-4ccc-b060-f18e34b38f4c<br>**Similarity:** 6.408306121826172<br>**Text:** Then will
arise Salman, Miqdad and AbuDharr."[39]

The Holy Prophet said, "Allah has ordered me to love four of my
companions."People asked who those four companions were. The Holy Prophet
said, "'Ali bin AbiTalib, Salman, Miqdad and Abu Dharr."[40]According to
traditions, Allah sent forSalman gifts and presents from Paradise; and the
Paradise eagerly awaited hisarrival.[41]

Once Mansur bin Buzurg, himself of Persian origin, asked Imam Ja'far al-Sadiq
as towhy he remembered Salman al-Farsi so much. The Imam said, "Do not say
'Salmanal-Farsi (the Persian)'. Say, 'Salman of Muhammad.' You should know
that the reasonof my often remembering him are three of his special virtues:
First, he discarded himown preferences in view of the preferences of Amirul
mu'minin 'Ali. Second, he lovedpoor and preferred them against rich and
wealthy persons. Third, he loved knowledgeand knowledgeable persons. Verily
Salman was a good servant of God, a pure Muslimand he was not from the
polytheists.<br>

**Node ID:** 7e803d93-6e97-4bc4-8686-71aafe7682dc<br>**Similarity:** 6.330111026763916<br>**Text:** ## Chapter 90 Q. 89: ABOUT SALMAN AL-FARSI (R.A.)

_Was Salman Al-Farsi (r.a.) governor of Mada'in at the time of his death? When
did he die and where?_

**A:** Salman (r.a.) was appointed governor of Mada'in by the 2nd caliph. The
caliph usually changed governors every two years. When he got the news that
Salman (r.a.) was about to enter Medina, the caliph went out with many
companions to welcome him. It was a very unusual honour paid to returning
governor. The reason became clear when they saw Salman (r.a.) returning with
the same robe in which he had set out. He had not taken from the treasury
anything for himself during his governorate.

Then Salman (r.a.) settled in Basrah, where he married a lady. After a few
years, he shifted with his family to Mada'in. I think he did so because
Mada'in people spoke Persian.

He died in Mada'in in the last year of 'Uthman's caliphate, i.e. in 36 A.H.
(Vide At-Tabaqat by Ibn Sa'd, vol.4, p.93).<br>

**Node ID:** 0f7de2b7-4a93-4842-a02c-b4af9530aa80<br>**Similarity:** 6.204136848449707<br>**Text:** After the martyrdom of this epitome of perfect faith, Hajjaaj suffered a
derangement of his senses and did not live for more than fifteen nights.
Before his death, he would occasionally lose consciousness, but regaining it,
he would repeatedly mutter:

“Why did I ever get involved with Sa’eed Ibn Jubair?”[15]127


### Salman Farsi’s Rank

  
Faith has ten ranks and Salman Farsi was on its tenth rank. He possessed
knowledge of the unseen, the ability to interpret dreams and misfortunes, was
well versed in genealogy and had even been favoured with gifts of Paradise, in
this world. The Holy Prophet (s.a.w.) said: “Whenever Jibraeel would descend,
he would, on behalf of God, say:  _Convey my salutation to Salman!_ ”

Here is an example of the high degree of Salman’s faith. Once, Abu Dharr paid
Salman a visit. Salman had placed a utensil over the fire to heat its
contents. The two men sat together for some time, talking to each other, when
suddenly the vessel toppled over, but to Abu Dharr’s amazement, its contents
did not spill. Salman, picking the utensil up, placed it back upon the fire. A
little later, the vessel fell over again, but once again its contents did not
spill. Again Salman picked it up and placed the vessel upright.

Amazed, Abu Dharr hurried out of Salman’s house and was lost in contemplation
over what he had witnessed. He happened to come across Amirul Mu'mineen (a.s.)
on the way and narrated the incident to him.

Having heard Abu Dharr’s narration, the Imam (a.s.)<br>

**Node ID:** 007843b4-9294-4b46-b312-8ed8c9ff02eb<br>**Similarity:** 6.181035041809082<br>**Text:** Thus,
in this light, anyone who believes in Islam and its Prophet must necessarily
declare his belief in the existence of the promised Mahdi who is now in
occultation.

Now we shall relate two ahadith on this issue from the famous Sunni book,
_Yanābi‘u 'l-Muwaddah_ :

1\. The Prophet said, “Al-Mahdi is a descendent of mine. He will be in
concealment, and when he manifests himself, the earth will become filled with
justice just as it was previously filled with injustice.”

2\. Salman al-Farsi says that he went to the Prophet when Husayn bin ‘Ali was
sitting in his lap and the Prophet was kissing his eyes and mouth, and then
the Prophet said to Husayn, “You are a chief, son of a chief, and the brother
of a chief; you are an Imam, son of an Imam, and the brother of an Imam. You
are the Proof (_hujjah_), son of a proof and the brother of a proof; you will
be the father of nine proofs of whom the ninth will be the  _qā'im_ (one who
will establish the divine rule on this earth).”


### 5\. The Length of al-Mahdi’s Life

We believe that the lengthening of a man's life is not an impossibility,
because it is explicitly stated in the Qur'ān that,**_“We sent Nuh to his
people and so he lived among them a thousand years, all but fifty.”_ (29:14)**
Even modern science does not deny the possibility of lengthening the human
lifespan.<br>

...

**Node ID:** 208e44a0-5359-46ef-ac45-0d0a2abf4687<br>**Similarity:** 4.418776988983154<br>**Text:** We were lights who glorified
Him, listened to Him and obeyed Him."

"O Messenger of Allah, may my father and mother be your ransom! What is there
for the person who recognizes these men as they should be recognized?"

"O Salman, whoever recognizes them as they should be recognized, and follows
their example, befriends them and is free of their enemies, by Allah! he is
one of us. He will return to where we return, and he will be where we are!"

"O Messenger of Allah, is there belief without knowing their names and
lineage?"

"No, Salman."

"Messenger of Allah, where will I find them?"

"You already know al-Husayn; then there will be the master of the worshippers,
'Ali ibn Husayn (Zayn al-'Abidin); then his son Muhammad ibn 'Ali, the piercer
of the knowledge of the early and the later prophets and messengers (al-
Baqir); then Ja'far ibn Muhammad, the truthful tongue of Allah (al-Sadiq);
then Musa ibn Ja'far, the one who kept his rage silent through patience in
Allah (al-Kadhim); then 'Ali ibn Musa, pleased with the secret of Allah (al-
Rida '); then Muhammad ibn 'Ali, the chosen one from the creatures of Allah
(al-Mukatar); then 'Ali ibn Muhammad, the guide to Allah (al-Hadi); then al-
Hasan, son of 'Ali, the silent, trustworthy guardian over the secret of Allah
(al-'Askari); then mim ha' mim dal (Muhammad), called Ibn al-Hasan, the
announcer who establishes the right of Allah."

Salman said, 'I wept.<br>

**Node ID:** 9db1f091-377d-48a1-a4a2-ffe29e49630a<br>**Similarity:** 4.415273189544678<br>**Text:** The subject: Was
Salman amuhajir or an ansar? The ansar claimed that as Salman came to the
Prophet inMedina, he belonged to the ansar group; the muhajirun claimed that
as Salman hadleft his home and family, he was a muhajir.This friendly dispute
also shows how great had become the status of Salman within ashort period of
three years that every group wanted to claim him as their own.Anyhow, the
dispute was referred to the highest authority - the Prophet, who decidedthat
Salman was from neither of the two groups; he said' "Salman minna Ahl al-Bayt-
Salman is from us, the family [of the Prophet]."[30] It was such a great
honour whichhas continuously been mentioned in traditions and poems. A poet
says:-

The devotion of Salman was his pedigree,while there was no relationship
between Noah and his son.

The Holy Prophet had also said, "Salman is a sea which cannot be exhausted and
atreasure which never comes to end. Salman is from us, the family [of the
Prophet]; hehas been given wisdom, and is bestowed with reason.<br>

**Node ID:** f7b22fd5-30d0-40de-9010-638a6642b286<br>**Similarity:** 4.389068603515625<br>**Text:** The second caliph described ibn 'Auf as
the “pharaoh of this nation.”[238] The very notion that only ten of the
Muslims should be guaranteed Paradise is illogical, since it exempts hundreds
of other sincere and virtuous Muslims, such as Hamzah - 'Sayyid al-shuhada
(Master of the Martyrs) and uncle of the Prophet, Ja'far ibn Abi Talib, Zayd
ibn Haritha, Sa'd ibn Ma'adh, 'Ammar ibn Yassir, about whom the Prophet said,
“He is filled with faith (iman) from head to toe,” and Salman al-Farsi, about
whom the Prophet said, “He is one of us, the Ahlul Bayt.” Since the hadith of
the ten who are guaranteed Paradise was narrated by Sa'id ibn Zayd, who was
not on good terms with the Ahlul Bayt, the Shi'a school of thought can not
accept it.


### Abu Hurayra

The man who narrated the largest number of hadith—5,374 (446 of which are in
Sahih al-Bukhari), although he says that he only spent three years with the
Prophet[239]—was Abu Hurayra al-Dusi. He embraced Islam on the seventh year
after the migration to Madina.<br>

**Node ID:** b938ff18-f3a7-4652-ba4b-473a7d8684e5<br>**Similarity:** 4.376296043395996<br>**Text:** Now, if Allah decides that such a
believer should come on the Day of Judgement clean from all blemishes and free
from all sins, then He may put him under punishment during the Barzakh period.
And it is this punishment which has been referred to in the Hadith of Imam
Jafer Sadiq (A) which was mentioned earlier:

“We will intercede on your behalf on the Day of Judgement; but, by Allah, I am
worried about you whilst you would be in Barzakh.”

On the other hand, if Allah decides to leave his case in suspense till
Qiyamat, then it will fall under 3rd group.

Anyhow, all these things depend upon the justice and mercy of Allah. We have
not been told enough to know all the details about all the groups.

It is about the first and second groups (as well as the last mentioned
believers who will be punished in Barzakh) that the Holy Prophet (S) has said:
“When one dies his Qiyamat starts!” In this sense the Barzakh is called
_“Qiyamat-e-Sughra”_ (the lesser Qiyamat)

The reader is now advised to read again the traditions given in lesson 17; so
that he may understand many references which might have remained a mystery at
first.

[21] Ibid, p.154-155.


## Chapter 23 Salman Farsi talks with a Dead Person

It seems proper to give here a short account of the conversation which Salman,
the Persian, had had with a dead person. It is a lengthy tradition, but of
great benefit.

Asbagh bin Nubata (r.a.)<br>

**Node ID:** af14a1d0-f58c-4cc0-8485-3454cf381094<br>**Similarity:** 4.365414142608643<br>**Text:** ## Chapter 13 Deviation

Six years passed after Uthman's Caliphate.

Gradually, the Caliph deviated from Islam and from our Master Muhammad's
behaviour.

Uthman appointed his blood relatives rulers over the cities. They were bad,
unjust persons.

For example, Uthman appointed al-Waleed bin Utbah, his half brother, ruler
over Kufa. Al-Waleed drank alcohol and went drunk to the mosque.

Uthman made Marwan bin Hakam the real ruler of the Muslim government. He
appointed the rulers and dismissed them. He dismissed the great companion
Salman al-Farsi from Mada-inn and appointed one of his blood relatives over
it.

Uthman also dismissed Saad bin Abi Wakkas from Kufa and appointed al-Waleed
bin Akabah over it.

Uthman spent the Muslims' money on his blood relatives from Bani Ummayyah and
left the poor to live in pain.<br>

In [30]:
import json

# create a dictionary to store the query and nodes data
output_data = {
    'original_question': question,
    # 'query_variations': query_variations,
    'nodes': []
}

# extract relevant information from each node
for node in retrieved_nodes:
    node_info = {
        'text': node.node.text,
        'score': float(node.score),  # convert to float for JSON serialization
        'metadata': node.node.metadata
    }
    output_data['nodes'].append(node_info)

# write to JSON file
with open('output/hybrid.json', 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

# Vector Retriever

In [85]:
vector_retriever = index.as_retriever(similarity_top_k=top_n)

In [42]:
retrieved_nodes = vector_retriever.retrieve(question)

In [39]:
len(retrieved_nodes)

50

In [ ]:
display_retrieved_nodes(retrieved_nodes)

# Hybrid Retriever with BM25 + Chroma

In [50]:
query_gen_prompt = """You are an AI language model assistant specializing in query expansion. Your task is to generate {num_queries} diverse versions of the given user question. These variations will be used to retrieve relevant documents from a vector database, helping to overcome limitations of distance-based similarity search.

Original question: {query}

Instructions:
1. Create {num_queries} unique variations of the original question.
2. Ensure each variation maintains the core intent of the original question.
3. Use different phrasings, synonyms, or perspectives for each variation.
4. Consider potential context or implications not explicitly stated in the original question.
5. Avoid introducing new topics or drastically changing the meaning of the question.

Please provide your {num_queries} question variations, each on a new line:
"""

In [ ]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    retrievers=[
        vector_retriever,
        bm25_retriever,
    ],
    retriever_weights=[0.6, 0.4],
    num_queries=10,
    query_gen_prompt=query_gen_prompt,
    use_async=True,
    similarity_top_k=top_n,
    mode="simple",
    verbose=True
)

In [68]:
retrieved_nodes = retriever.retrieve(question)

Generated queries:
1. What is the lineage of Salman Al Farsi?
2. From whom does Salman Al Farsi descend?
3. Who is the father of Salman Al Farsi?
4. Whose son is Salman Al Farsi known to be?
5. To whom is Salman Al Farsi related as a son?
6. What is the paternal ancestry of Salman Al Farsi?
7. Who is Salman Al Farsi's parent?
8. Which family does Salman Al Farsi belong to as a son?
9. Who is the progenitor of Salman Al Farsi?


In [69]:
len(retrieved_nodes)

50

In [70]:
display_retrieved_nodes(retrieved_nodes)

**Node ID:** b4220de6-90dd-48d8-b8c8-e51d95f8c75e<br>**Similarity:** 9.112570762634277<br>**Text:** ![Cover](images/cover.png)


![IslamicMobility](images/logo-feedbooks-tiny.png)

# Salman Al Muhammadi

## Kamal al-Syyed - XKP

  
**Published:** 2012  
**Tag(s):** companion prophet ali imam shia shiite shiaism husain salman farsi
persian iran syria holy xkp madina makka pagan madain kufa


## Chapter 1 The Son of Islam

It was before noon. Some Muslims sat in Prophet's Mosque waiting for Azan
(call to prayer) to say noon prayer.

Salman entered mosque and greeted his believing brothers.

The Muslims wanted to know the Persian man's tribe. They mentioned their
tribes loudly to let Salman hear them.

One of them said:

_I belong to Tamim's tribe._

Another said:

_I belong to the Quraish._

A third said:

_I belong to al-Aus tribe._

And so on.

But Salman was silent. They wanted to know his tribe. So they asked him:

_Salman, where are you from?_

To teach them the meaning of Islam, Salman answered:

_I'm the son on Islam! I was lost! So, Allah's guided me with Muhammad. I was
poor! So, Allah's made me rich with Muhammad. I was a slave! So, Allah's
released me with Muhammad. This is my tribe!_

The Muslims in mosque kept silent because Salman taught them a lesson of
Islam.<br>

**Node ID:** 55cc8de7-14f8-4053-9df7-1b813664f413<br>**Similarity:** 7.65976619720459<br>**Text:** There too Abu Dharr bore witness about Allah
and his Prophet (a.s) thrice.

Now Hazrat Ali (a.s) took Abu Dharr to the presence of the Prophet (a.s). He
greeted the Prophet (a.s) who kindly asked him to sit and bear witness to the
unity of Allah and that Mohammed (a.s) is Allah’s Prophet. Now the Prophet
(a.s) said, “O Abu Dharr! You must immediately return to your place that your
paternal cousin has expired and you are the soul inheritor of his estate. Go
home and take charge of the affairs of your estate. You must join us when we
summon you.”Therefore after the migration (_Hegira)_ Abu Dharr had the honour
of meeting the Prophet (a.s) in Medina.


## Chapter 4 SALMAN ACCEPTS ISLAM

Hazrat Imam Mohammed Baqir (a.s) says that some companions visited the grave
of the Prophet (a.s) one day and were busy conversing with each other. Hazrat
Ali (a.s) asked Salman Farsi, “O Salman! How did you accept the Faith of
Islam?” Salman replied, “O Ameer al Momineen! Since it is obligatory on me to
reply to your queries, I am narrating to you the circumstances under which I
accepted the Faith. I am a native of the city of Shiraz and my profession was
farming. My parents used to love me very much.<br>

**Node ID:** 7e803d93-6e97-4bc4-8686-71aafe7682dc<br>**Similarity:** 7.643820762634277<br>**Text:** ## Chapter 90 Q. 89: ABOUT SALMAN AL-FARSI (R.A.)

_Was Salman Al-Farsi (r.a.) governor of Mada'in at the time of his death? When
did he die and where?_

**A:** Salman (r.a.) was appointed governor of Mada'in by the 2nd caliph. The
caliph usually changed governors every two years. When he got the news that
Salman (r.a.) was about to enter Medina, the caliph went out with many
companions to welcome him. It was a very unusual honour paid to returning
governor. The reason became clear when they saw Salman (r.a.) returning with
the same robe in which he had set out. He had not taken from the treasury
anything for himself during his governorate.

Then Salman (r.a.) settled in Basrah, where he married a lady. After a few
years, he shifted with his family to Mada'in. I think he did so because
Mada'in people spoke Persian.

He died in Mada'in in the last year of 'Uthman's caliphate, i.e. in 36 A.H.
(Vide At-Tabaqat by Ibn Sa'd, vol.4, p.93).<br>

**Node ID:** 007843b4-9294-4b46-b312-8ed8c9ff02eb<br>**Similarity:** 7.602144718170166<br>**Text:** Thus,
in this light, anyone who believes in Islam and its Prophet must necessarily
declare his belief in the existence of the promised Mahdi who is now in
occultation.

Now we shall relate two ahadith on this issue from the famous Sunni book,
_Yanābi‘u 'l-Muwaddah_ :

1\. The Prophet said, “Al-Mahdi is a descendent of mine. He will be in
concealment, and when he manifests himself, the earth will become filled with
justice just as it was previously filled with injustice.”

2\. Salman al-Farsi says that he went to the Prophet when Husayn bin ‘Ali was
sitting in his lap and the Prophet was kissing his eyes and mouth, and then
the Prophet said to Husayn, “You are a chief, son of a chief, and the brother
of a chief; you are an Imam, son of an Imam, and the brother of an Imam. You
are the Proof (_hujjah_), son of a proof and the brother of a proof; you will
be the father of nine proofs of whom the ninth will be the  _qā'im_ (one who
will establish the divine rule on this earth).”


### 5\. The Length of al-Mahdi’s Life

We believe that the lengthening of a man's life is not an impossibility,
because it is explicitly stated in the Qur'ān that,**_“We sent Nuh to his
people and so he lived among them a thousand years, all but fifty.”_ (29:14)**
Even modern science does not deny the possibility of lengthening the human
lifespan.<br>

**Node ID:** 0f7de2b7-4a93-4842-a02c-b4af9530aa80<br>**Similarity:** 7.583582878112793<br>**Text:** After the martyrdom of this epitome of perfect faith, Hajjaaj suffered a
derangement of his senses and did not live for more than fifteen nights.
Before his death, he would occasionally lose consciousness, but regaining it,
he would repeatedly mutter:

“Why did I ever get involved with Sa’eed Ibn Jubair?”[15]127


### Salman Farsi’s Rank

  
Faith has ten ranks and Salman Farsi was on its tenth rank. He possessed
knowledge of the unseen, the ability to interpret dreams and misfortunes, was
well versed in genealogy and had even been favoured with gifts of Paradise, in
this world. The Holy Prophet (s.a.w.) said: “Whenever Jibraeel would descend,
he would, on behalf of God, say:  _Convey my salutation to Salman!_ ”

Here is an example of the high degree of Salman’s faith. Once, Abu Dharr paid
Salman a visit. Salman had placed a utensil over the fire to heat its
contents. The two men sat together for some time, talking to each other, when
suddenly the vessel toppled over, but to Abu Dharr’s amazement, its contents
did not spill. Salman, picking the utensil up, placed it back upon the fire. A
little later, the vessel fell over again, but once again its contents did not
spill. Again Salman picked it up and placed the vessel upright.

Amazed, Abu Dharr hurried out of Salman’s house and was lost in contemplation
over what he had witnessed. He happened to come across Amirul Mu'mineen (a.s.)
on the way and narrated the incident to him.

Having heard Abu Dharr’s narration, the Imam (a.s.)<br>

...

**Node ID:** 4c190b82-3535-4a9e-80a7-3f56f7826b45<br>**Similarity:** 5.746349334716797<br>**Text:** ### The Humbleness of Salman Farsi

Salman had been the governor of one of the cities of Syria for some time. His
conduct during the period of his rule remained unchanged from that before his
governorship; he would always wear ordinary clothing, travel on foot and even
place his household things as security and surety (for borrowing money).

Once, while moving through the bazar, he saw a man who had purchased some
alfalfa and was looking for someone to carry it to his house for him. Salman
approached the person, who failed to recognize him, and agreed to carry his
load free of charge. The man placed the load of alfalfa on Salman’s back. As
they were walking, they came across a person who immediately recognized
Salman.

“O’ Leader! Where are you carrying this burden?” he exclaimed.

Hearing these words, the owner of the load realized that the person carrying
his load was Salman. He fell down on his knees and implored: “Forgive me, for
I had failed to recognise you.”  
“Nevertheless, I must carry this load to your house,” said Salman.

When he had done so, he said to the man, “I have fulfilled my promise; now it
is for you to promise that henceforth you will never seek the services of
anyone for anything. (And know!) By you carrying things which you are able to,
it will not reflect negatively upon your manliness.”[4]<br>

**Node ID:** ec0007a7-ea1d-4fa3-94c0-df548795dd91<br>**Similarity:** 5.7379608154296875<br>**Text:** ### The Fearful Youth

Salman Farsi was passing through the blacksmiths’ market of Kufah when he
observed that a crowd had gathered around a youth who lay on the ground,
senseless. When the people saw Salman, they requested him to recite a
supplication so that the youth could come out of his unconsciousness.

As soon as Salman came closer, the youth got up and said: “There is nothing
wrong with me. It is just that I was passing through this market when I
noticed the blacksmiths striking metal with their iron hammer and this made me
recall what Allah has said in the Qur’an: “In addition there will be maces of
iron (to punish) them.”[4](http://www.al-islam.org/anecdotes-reflection-
part-2-sayyid-ali-akbar-sadaaqat/36-fear#footnote4_ol64mg8)As this verse
crossed my mind, I was overcome by this state.”

Salman became interested in the youth, grew fond of him and made him his
brother. They were always friends with each other till one day the youth fell
ill and was almost on the verge of dying. Salman sat down near his head and
then, addressing Izra’eel, said: “O’ Izra’eel! Be tolerant and lenient towards
my young brother and be kind and gentle to him!”

“O’ Servant of Allah! I am the friend of all the believers and kind to them
all,” Izra`il replied.[5]<br>

**Node ID:** 3a26161c-9fa7-426c-a839-497e085e79d3<br>**Similarity:** 5.729222297668457<br>**Text:** **2768.** If the heirs of a dead person are his paternal uncles and paternal
aunts, out of whom some are the real brothers and sisters of his father and
others are paternal or maternal brothers and sisters of his father, the
paternal uncles and paternal aunts of the dead person, who are paternal
brothers and paternal sisters of his father, do not inherit anything.

And the well-known view is that if the dead person has a paternal uncle or a
paternal aunt, who are the maternal brother and sister of his father, the
property is divided into 6 parts, out of which one part is taken by the
paternal uncle or maternal aunt of the deceased, and the remaining is taken by
the dead person's paternal uncles and paternal aunts, and, in case the dead
person has no real paternal uncles and real paternal aunts, the remaining 5
parts are also taken by the paternal uncles and paternal aunts of the
deceased, who are the maternal brothers or sisters of his father.

And if the dead person happens to have also paternal uncles and paternal
aunts, who are the maternal brothers and sisters of his father, the property
is divided into 3 parts, out of which 2 parts are taken by the real paternal
uncles and real paternal aunts of the dead person and in case there are no
such uncles and aunts, they are taken by those paternal uncles and paternal
aunts of the dead person who are the paternal brothers and sisters of his
father, and one part is taken by those paternal uncles and paternal aunts of
the dead person who are the maternal brothers and sisters of his father.

It is not, however.<br>

**Node ID:** d41182b7-1d08-49d2-9ab2-41c9aff7417f<br>**Similarity:** 5.712906360626221<br>**Text:** If this son were not a believer as Noah saw him, he would not have called him to board the Ark at all. For it was Noah who had explicitly called for the disbelievers to be purged from the earth when he said: *‘My Lord! Do not leave on the earth any inhabitant from among the faithless!’* The fact that he said this while calling his son to board the Ark shows that Noah considered this son to be a believer. His disobedience to his father’s command to board the Ark was not itself an act of disbelief (*kufr*) nor tantamount to it, but was merely a sin (*ma‘ṣiya*), which falls short of total disbelief.

It was in light of this that Noah said, *‘My Lord! My son is indeed from my family. Your promise is indeed true* (wa inna wa‘daka al-ḥaqq) ...’—he recalled the promise of his Lord to deliver his family and added that his son was part of his family. The fact that he invokes God as *‘Lord’ (rabb)* indicates that this is an appeal for divine mercy; the fact that he uses the possessive pronoun *‘my’* in *‘my son’* and *‘my family’* is to demonstrate his argument; while the use of the preposition *inna* (‘indeed’) and the definite article *(al-)* in *Your promise is indeed true* show that his faith in God is genuine.

Now, placing the two statements, *‘My son is indeed from my family’* and *‘indeed, Your promise is true’*, side-by-side, imply that his son should have survived the flood. However, note that Noah, as a divine servant, shows humility towards God by not explicitly drawing this conclusion. Indeed, judgement belongs only to God.<br>

**Node ID:** 9acafc1d-8f1b-46dd-9621-1ab76e95a969<br>**Similarity:** 5.705635070800781<br>**Text:** **2768.** If the heirs of deceased are his paternal uncles and paternal aunts,
some of whom are the real brothers and sisters of his father, while others are
paternal or maternal half brothers and sisters of his father, those who are
paternal half brothers and sisters will not inherit anything.  
And if the deceased is also survived by one paternal uncle or one paternal
aunt, who are the maternal half brother and half sister of his father, the
estate will be divided into 6 parts, from which one part will be taken by the
paternal uncle or paternal aunt of the deceased, and the remaining will be
taken by the full real paternal uncles and paternal aunts of the deceased.  
If the deceased has no real full paternal uncles and real full paternal aunts,
the remaining 5 parts will be taken by those paternal uncles and paternal
aunts of the deceased who are the paternal half brothers or sisters of his
father.  
But, if the deceased happens to have those paternal uncles together with
paternal aunts who are the maternal half brothers and sisters of his father,
the estate will be divided into 3 parts, from which 2 parts will be taken by
the real paternal uncles and real paternal aunts of the deceased, who are half
paternal brothers and sisters of his father.  
Then the remaining one part will be taken by those paternal uncles and
paternal aunts of the deceased person, who are the maternal half brothers and
sisters of his father.<br>